In [18]:
from initial import *

import pandas as pd

In [2]:
# from gamma import gamma_api

# gamma_api()

## 下载原始event数据

In [3]:
# 目标 URL
url = "https://gamma-api.polymarket.com/events?active=true&closed=false&archived=false&limit=1000000000000000000"
# url = 'https://gamma-api.polymarket.com/events?closed=false'

# 发送 GET 请求
response = requests.get(url)
response.raise_for_status()  

# 将返回的 JSON 数据解析成 Python 对象
data = response.json()


if isinstance(data, list):
    df = pd.DataFrame(data)
elif isinstance(data, dict):
    key = "events"  
    if key in data:
        df = pd.DataFrame(data[key])
    else:
        df = pd.DataFrame(data)
else:
    raise ValueError("未知的数据结构")

df_market = df[['id','title','markets']]

df_market

,id,title,markets
0,12483,Premier League Winner,"[{'id': '506728', 'question': 'Manchester City..."
1,12585,Champions League Winner,"[{'id': '507300', 'question': 'Will Inter Mila..."
2,12672,La Liga Winner,"[{'id': '507395', 'question': 'Will Real Madri..."
3,12756,English Premier League Top Scorer,"[{'id': '507683', 'question': 'Will Luis Diaz ..."
4,12815,NBA Champion,"[{'id': '507869', 'question': 'Will the LA Cli..."
...,...,...,...
495,17728,Meloni out as Prime Minister of Italy before J...,"[{'id': '521535', 'question': 'Meloni out as P..."
496,17729,Meloni out as Prime Minister of Italy in 2025?,"[{'id': '521536', 'question': 'Meloni out as P..."
497,17731,Will Trump end USAID in first 100 days?,"[{'id': '521540', 'question': 'Will Trump end ..."
498,17738,Eurovision Winner 2025,"[{'id': '521568', 'question': 'Will Albania wi..."


## 提取 ID，title，markets，volume24hr_list

In [4]:
market_volumes = []

for _, row in df_market.iterrows():
    each_market = row['markets']
    
    if isinstance(each_market, list):  # Ensure it’s a list
        volumes = [(
                    entry.get('question', 'Unknown'), 
                    entry.get('volume24hrClob', 'Unknown'),
                    entry.get('clobTokenIds', 'Unknown'),
                    entry.get('conditionId', 'Unknown'), 
                    entry.get('spread', 'Unknown'),
                    entry.get('startDate','Unknown'),
                    entry.get('endDate','Unknown')
                    )
                    for entry in each_market if isinstance(entry, dict)]
        market_volumes.append(volumes)
    else:
        market_volumes.append([])  # Handle cases where 'markets' is not a list

df_market['volume24hr_spread_list'] = market_volumes

df_market

/var/folders/bz/pyhbjdvx2d708kdlz50s5cj80000gn/T/ipykernel_18394/607041685.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_market['volume24hr_spread_list'] = market_volumes


,id,title,markets,volume24hr_spread_list
0,12483,Premier League Winner,"[{'id': '506728', 'question': 'Manchester City...","[(Manchester City wins the Premier League?, 60..."
1,12585,Champions League Winner,"[{'id': '507300', 'question': 'Will Inter Mila...",[(Will Inter Milan win the UEFA Champions Leag...
2,12672,La Liga Winner,"[{'id': '507395', 'question': 'Will Real Madri...","[(Will Real Madrid win La Liga?, 6423.187042, ..."
3,12756,English Premier League Top Scorer,"[{'id': '507683', 'question': 'Will Luis Diaz ...",[(Will Luis Diaz be the top goalscorer in the ...
4,12815,NBA Champion,"[{'id': '507869', 'question': 'Will the LA Cli...",[(Will the LA Clippers win the 2025 NBA Finals...
...,...,...,...,...
495,17728,Meloni out as Prime Minister of Italy before J...,"[{'id': '521535', 'question': 'Meloni out as P...",[(Meloni out as Prime Minister of Italy before...
496,17729,Meloni out as Prime Minister of Italy in 2025?,"[{'id': '521536', 'question': 'Meloni out as P...",[(Meloni out as Prime Minister of Italy in 202...
497,17731,Will Trump end USAID in first 100 days?,"[{'id': '521540', 'question': 'Will Trump end ...","[(Will Trump end USAID in first 100 days?, 121..."
498,17738,Eurovision Winner 2025,"[{'id': '521568', 'question': 'Will Albania wi...","[(Will Albania win Eurovision 2025?, 16252.803..."


## 基于df_market，提取每个market的yes和no的token id

In [5]:
# Step 2: 创建一个新的 DataFrame，
# 其中包含满足条件的交易量事件及其 market id，同时添加 spread 列
filtered_data = [
    {
        'id': row['id'], 
        'question': entry[0], 
        'volume24hrClob': entry[1],
        'TokenID': entry[2],
        'conditionId': entry[3],
        'spread': entry[4],
        'startDate': entry[5],
        'endDate': entry[6]
    }
    for _, row in df_market.iterrows()
    for entry in row['volume24hr_spread_list']
    # 如果需要过滤：if lower_threshold <= entry[1] <= upper_threshold
]

df_filtered_volume = pd.DataFrame(filtered_data)
df_filtered_volume

,id,question,volume24hrClob,TokenID,conditionId,spread,startDate,endDate
0,12483,Manchester City wins the Premier League?,60815.799498,"[""88643218703041538921477211574402135868068849...",0xeba50b6a5e1a5a682c8aaf34152d4eb91e0410d59117...,0.001,2024-09-09T20:24:52.495018Z,2025-05-25T12:00:00Z
1,12483,Arsenal wins the Premier League?,23487.393004,"[""73195466598131388170821776880758003788512546...",0x320045be25e331375755d9126cbe20a319ace7c7d925...,0.003,2024-09-09T20:28:43.533604Z,2025-05-25T12:00:00Z
2,12483,Aston Villa wins the Premier League?,2707836.620939,"[""83156492313921775878087837698449837132415407...",0x951ead358ab84c4314b8a619593471d80c687cc02720...,0.001,2024-09-09T20:30:02.775002Z,2025-05-25T12:00:00Z
3,12483,Bournemouth wins the Premier League?,211367.9715,"[""20043121189468219599437255198027106640631597...",0xd35868afa9257d08411f4c7d61601213fc3d9fa1ebbc...,0.001,2024-09-09T20:30:39.18419Z,2025-05-25T12:00:00Z
4,12483,Everton wins the Premier League?,122006.69,"[""28179090282093807467382718742019031223670663...",0xa5bfa29f27399fce767abbff2d875bb5ad7a236748fb...,0.001,2024-09-09T20:55:43.375953Z,2025-05-25T12:00:00Z
...,...,...,...,...,...,...,...,...
2147,17738,Will Netherlands win Eurovision 2025?,2103.306278,"[""81494113250070000853911226027907208048734840...",0x677826754ff7328696c121a398fdf495dcd2e04f84ec...,0.011,2025-02-03T21:18:58.548658Z,2025-05-17T12:00:00Z
2148,17738,Will Spain win Eurovision 2025?,879.220239,"[""52792753704538441909316018819342296081865500...",0xae5e043ee3cec6bf13db32898b940f86c459c8f0509d...,0.003,2025-02-03T21:25:42.900818Z,2025-05-17T12:00:00Z
2149,17738,Will Switzerland win Eurovision 2025?,1498.05732,"[""64476134626186650599378715641194359140162587...",0x499a8d119a710062e6d56a3db97e33331a3cad02f9de...,0.008,2025-02-03T21:27:03.170701Z,2025-05-17T12:00:00Z
2150,17738,Will Country B win Eurovision 2025?,0,"[""74056828587204375778598920855056272422355956...",0xf07285f2c7092abbd3285969d483659d9c4b244eb13f...,1.000,2025-02-03T21:30:37.684413Z,2025-05-17T12:00:00Z


## 提取yes和no的token id, spread, market id

In [6]:
import ast
import pandas as pd

def extract_tokens(token_str):
    try:
        tokens = ast.literal_eval(token_str)
        if isinstance(tokens, list):
            yes_token = str(tokens[0]) if len(tokens) > 0 and tokens[0] is not None else None
            no_token = str(tokens[1]) if len(tokens) > 1 and tokens[1] is not None else None
        else:
            yes_token, no_token = None, None
        return [yes_token, no_token]
    except Exception:
        return [None, None]

# 示例：假设 df_filtered_volume 已存在且包含 'TokenID' 列
# 拆分为两个新列，并确保原始的 TokenID 列也被转换为列表，每个元素均为字符串
df_filtered_volume[['yes_token_id', 'no_token_id']] = df_filtered_volume['TokenID'].apply(lambda x: pd.Series(extract_tokens(x)))
df_filtered_volume['TokenID'] = df_filtered_volume['TokenID'].apply(lambda x: extract_tokens(x))

df_filtered_volume

,id,question,volume24hrClob,TokenID,conditionId,spread,startDate,endDate,yes_token_id,no_token_id
0,12483,Manchester City wins the Premier League?,60815.799498,[886432187030415389214772115744021358680688493...,0xeba50b6a5e1a5a682c8aaf34152d4eb91e0410d59117...,0.001,2024-09-09T20:24:52.495018Z,2025-05-25T12:00:00Z,8864321870304153892147721157440213586806884930...,5202270048105592915861981052115390705541146660...
1,12483,Arsenal wins the Premier League?,23487.393004,[731954665981313881708217768807580037885125462...,0x320045be25e331375755d9126cbe20a319ace7c7d925...,0.003,2024-09-09T20:28:43.533604Z,2025-05-25T12:00:00Z,7319546659813138817082177688075800378851254626...,2380758099274361309614515288149550771276495124...
2,12483,Aston Villa wins the Premier League?,2707836.620939,[831564923139217758780878376984498371324154078...,0x951ead358ab84c4314b8a619593471d80c687cc02720...,0.001,2024-09-09T20:30:02.775002Z,2025-05-25T12:00:00Z,8315649231392177587808783769844983713241540781...,4124576035317661748767784573536443551908988807...
3,12483,Bournemouth wins the Premier League?,211367.9715,[200431211894682195994372551980271066406315978...,0xd35868afa9257d08411f4c7d61601213fc3d9fa1ebbc...,0.001,2024-09-09T20:30:39.18419Z,2025-05-25T12:00:00Z,2004312118946821959943725519802710664063159781...,7530822654051987323923362314024017098865350041...
4,12483,Everton wins the Premier League?,122006.69,[281790902820938074673827187420190312236706635...,0xa5bfa29f27399fce767abbff2d875bb5ad7a236748fb...,0.001,2024-09-09T20:55:43.375953Z,2025-05-25T12:00:00Z,2817909028209380746738271874201903122367066354...,2156099967661373030229856509463824941534153655...
...,...,...,...,...,...,...,...,...,...,...
2147,17738,Will Netherlands win Eurovision 2025?,2103.306278,[814941132500700008539112260279072080487348400...,0x677826754ff7328696c121a398fdf495dcd2e04f84ec...,0.011,2025-02-03T21:18:58.548658Z,2025-05-17T12:00:00Z,8149411325007000085391122602790720804873484000...,1054070496265651101341410650835420184340310728...
2148,17738,Will Spain win Eurovision 2025?,879.220239,[527927537045384419093160188193422960818655008...,0xae5e043ee3cec6bf13db32898b940f86c459c8f0509d...,0.003,2025-02-03T21:25:42.900818Z,2025-05-17T12:00:00Z,5279275370453844190931601881934229608186550080...,6235542079394529570845635347886836642893869640...
2149,17738,Will Switzerland win Eurovision 2025?,1498.05732,[644761346261866505993787156411943591401625879...,0x499a8d119a710062e6d56a3db97e33331a3cad02f9de...,0.008,2025-02-03T21:27:03.170701Z,2025-05-17T12:00:00Z,6447613462618665059937871564119435914016258799...,1096298881665621869806083829548837003303037782...
2150,17738,Will Country B win Eurovision 2025?,0,[740568285872043757785989208550562724223559569...,0xf07285f2c7092abbd3285969d483659d9c4b244eb13f...,1.000,2025-02-03T21:30:37.684413Z,2025-05-17T12:00:00Z,7405682858720437577859892085505627242235595694...,2497704293424763353182667784629174973329306617...


## 设置threshold过滤事件

In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def calculate_threshold(idx, row):
    volume24h = row['volume24hrClob']
    yes_token = row["yes_token_id"]
    threshold = None
    try:
        # 获取订单簿数据
        order_book_yes = client.get_order_book(yes_token)
        
        # 计算最佳买价和最佳卖价
        best_bid_yes = max(float(order.price) for order in order_book_yes.bids)
        best_ask_yes = min(float(order.price) for order in order_book_yes.asks)
        
        size_at_best_bid_price = next(float(order.size) for order in order_book_yes.bids if float(order.price) == best_bid_yes)
        size_at_best_ask_price = next(float(order.size) for order in order_book_yes.asks if float(order.price) == best_ask_yes)
        
        threshold = volume24h / (size_at_best_bid_price + size_at_best_ask_price)
    except Exception as e:
        pass

    # 返回时保留原始索引，方便后续排序或合并
    new_row = dict(row)
    new_row["threshold"] = threshold
    new_row["_idx"] = idx  # 添加原始索引
    
    return new_row

filtered_rows = []
with ThreadPoolExecutor(max_workers=30) as executor:

    # 提交任务时同时传入索引和值
    futures = [executor.submit(calculate_threshold, idx, row_data) for idx, row_data in df_filtered_volume.iterrows()]
    
    # as_completed(futures)：这是一个生成器，它会按照任务完成的顺序依次返回 Future 对象，而不一定按照原始提交顺序。
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Order Books"):
        # print(future.result())
        filtered_rows.append(future.result())

# 将结果转换为 DataFrame
results_df = pd.DataFrame(filtered_rows)

# 根据原始索引排序
results_df.sort_values(by="_idx", inplace=True)



# 把 threshold 加入到 df_filtered_volume
df_filtered_volume["threshold"] = results_df["threshold"].values

df_filtered_volume.sort_values(by="threshold", ascending=False, inplace=True)

Processing Order Books: 100%|██████████| 2152/2152 [05:37<00:00,  6.38it/s]


In [15]:
# 转换日期列，无法解析的将变为 NaT
df_filtered_volume['startDate'] = pd.to_datetime(df_filtered_volume['startDate'], errors='coerce')
df_filtered_volume['endDate'] = pd.to_datetime(df_filtered_volume['endDate'], errors='coerce')


# 获取今天的日期（带utc时区）
today = pd.Timestamp.today(tz='UTC')

# 筛选符合 threshold、spread 和时间差大于7天条件的记录
df_top_10_threshold = df_filtered_volume[
    (df_filtered_volume['threshold'] >= df_filtered_volume['threshold'].quantile(0.95)) & # volume 前 24 h 最高的top percentile
    (df_filtered_volume['spread'] >= 0.05) & # spread 大于 。。。。
    ((df_filtered_volume['endDate'] - today) > pd.Timedelta(days=30)) # 距离事件结束大于n天
].sort_values(by='threshold', ascending=False).reset_index(drop=True)

# 构造 question 到 TokenID、conditionId 和 spread 的字典
question_to_tokenid = df_top_10_threshold.set_index("question")[["TokenID", "conditionId", "spread"]].to_dict("index")

question_to_tokenid

{'Will Bitcoin reach $110,000 by December 31, 2025?': {'TokenID': ['39044137795224187219385013942783323059681835335864307732029139183983670488070',
   '13660516767041361627171788514669314634677119276162516861586896150850746376938'],
  'conditionId': '0x63e3c9432d9e3a0e9665d1152fc0a05f6ed17c9033eb9290062d25d2afb2bed5',
  'spread': 0.075},
 'Lori Chavez-DeRemer confirmed as Labor Secretary?': {'TokenID': ['73476233433788166151364812284559234561295115610106207256844083240142263313569',
   '62681245476175148357326333306961103580658264122355426352561031820311708934423'],
  'conditionId': '0x663bdcf8373cfa2af610ee93ff43f4451c0d0d07322129a6432b8127884687fd',
  'spread': 0.056},
 'Dan Clancy out as Twitch CEO in 2025?': {'TokenID': ['52521422646418218239213208751328222694757717504934616636622590621075679371215',
   '66390163516315956466696118534227657995267232292264828573695652317434995942865'],
  'conditionId': '0x08a5f56fdeacd0d3adbc618d12f73927d51a5654fd3883fad95ed863209deda9',
  'spread': 